Before you run any of the code below, please make sure that you have installed the necessary packages using the line of code below

In [1]:
#May need to install programmes in command prompt first with:
#pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org numpy
#pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org plotly (before dash if you get an error)
#pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org openpyxl
#pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org matplotlib (etc......)
#To run in browser need to use this web address http://127.0.0.1:2234/ (2234 you need to take from the port number at the end of this block of code after you run it) this will vary for each line of code below 


ALSO PLEASE NOTE THAT IF ANYTIME THE CODE SEEMS TO BE LAGGING WHEN YOU OPEN THE FOLLOWING BROWSER LINK ABOVE, PLEASE TRY RERUNNING THE CODE ON VSCODE THEN OPENING THE BROWSER AGAIN. ALSO PLEASE MAKE A SEPERATE COPY OF THIS CODE IN YOUR OWN FOLDER AND $$DO NOT$$ TRY TO RUN THIS CODE IN THE SAME FILE THAT IT IS STORED IN (ie Rheo Mystery FOLDER). IF ANY OTHER ISSUES WITH CODE PLEASE CONSULY LIAM RATCLIFFE 

WHERE TO GET THE DATA:
1) *OSCILLATORY RHEOLOGY* - Export data from the rheocompass software, refer to manual of oscillatory rheology technique for ice cream premixes on how to do this. Format for how the data in the table should look like is in the same file 
2) *ROTATIONAL RHEOLOGY* - Export data from core measurement data capture, link here https://unilever.sharepoint.com/sites/MastersizerDataCapture/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2FMastersizerDataCapture%2FShared%20Documents%2FData%2Frheology&viewid=a285c3e2%2D1131%2D4b23%2Db8fe%2D87166b191f9a. How data should look like is in the same file. 
3) *LASER DIFFRACTION* - Export data from powerbi core measurement dashboard, please note you will need to gain access to this and Erin Saunders is the main contact. (README FILE SHOWS YOU HOW TO GET THE DATA). LINK here https://app.powerbi.com/groups/me/reports/758f8046-0604-4bc1-9b96-759621caf169/ReportSectioncf78ad36fc49107a38f8?experience=power-bi&bookmarkGuid=Bookmark278ca0407fdd9fa970db. How data shoudl look like in same file. 

# ROTATIONAL RHEOLOGY 

In [ ]:
# Import necessary packages
import pandas as pd
import numpy as np
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, ALL
import dash_daq as daq
import base64
import io
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib

# Initialize the app
app = Dash(__name__)

# Color and marker options
color_options=[]
for name, hex in matplotlib.colors.cnames.items():
    color_options.append(name)

font_options = ['Arial', 'Helvetica', 'Times New Roman', 'Courier New', 'Verdana', 'Georgia', 'Palatino', 'Garamond', 'Comic Sans MS', 'Impact']

# App layout
app.layout = html.Div([
    html.H1("Rotational rheology Data Analysis", style={'textAlign': 'center'}),
    dcc.Upload(
        id='upload-data',
        children=html.Button('Upload Excel File', style={'margin': '10px'}),
        multiple=False
    ),
    dcc.Dropdown(id='sheet-dropdown', placeholder="Select Sheet", style={'margin': '10px'}),
    dcc.Dropdown(id='test-dropdown', placeholder="Select Tests", multi=True, style={'margin': '10px'}),
    dcc.Dropdown(id='color-picker', options=[{'label': color, 'value': color} for color in color_options],
                 placeholder="Select Plot Color", multi=True, style={'margin': '10px'}),
    dcc.Dropdown(id='font-picker', options=[{'label': font, 'value': font} for font in font_options],
                 placeholder="Select Font Type", style={'margin': '10px'}),
    html.Div([
        dcc.Input(id='marker-size', type='number', placeholder='Marker Size', style={'margin': '10px'}),
    ]),
    html.Div([
        dcc.Input(id='x-min', type='number', placeholder='X-axis Min', style={'margin': '10px'}),
        dcc.Input(id='x-max', type='number', placeholder='X-axis Max', style={'margin': '10px'}),
        dcc.Input(id='y-min', type='number', placeholder='Y-axis Min', style={'margin': '10px'}),
        dcc.Input(id='y-max', type='number', placeholder='Y-axis Max', style={'margin': '10px'}),
        dcc.Checklist(
            id='gridlines-toggle',
            options=[{'label': 'Show Gridlines', 'value': 'show'}],
            value=['show'],
            style={'margin': '10px'}
        )
    ]),
    html.Div(id='plot-labels-div'),
    html.Button('Add Plot Label', id='add-plot-label', n_clicks=0, style={'margin': '10px'}),
    html.Div([
        dcc.Input(id='font-size', type='number', placeholder='Font Size', style={'margin': '10px'}),
    ]),

    dash_table.DataTable(id='data-table', page_size=10, style_table={'overflowX': 'auto'}),
    html.Div(id='plot-img')
])

# Callback to add input fields for plot labels
@callback(
    Output('plot-labels-div', 'children'),
    Input('add-plot-label', 'n_clicks'),
    State('plot-labels-div', 'children')
)
def add_plot_labels(n_clicks, children):
    if children is None:
        children = []
    new_input = dcc.Input(id={'type': 'plot-label', 'index': n_clicks}, type='text', placeholder=f'Plot Label {n_clicks}', style={'margin': '10px'})
    children.append(new_input)
    return children

# Add controls to build the interaction
@callback(
    Output('sheet-dropdown', 'options'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_sheet_dropdown(contents, filename):
    if contents is None:
        return []

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.ExcelFile(io.BytesIO(decoded))
    sheet_names = df.sheet_names
    return [{'label': sheet, 'value': sheet} for sheet in sheet_names]

@callback(
    Output('test-dropdown', 'options'),
    Input('sheet-dropdown', 'value'),
    State('upload-data', 'contents')
)
def update_test_dropdown(sheet_name, contents):
    if sheet_name is None or contents is None:
        return []

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df_tests = pd.read_excel(io.BytesIO(decoded), sheet_name=sheet_name, usecols=['Application:', 'Anton Paar RheoCompass™ V1.22.0.0'])
    df_tests['Application:'] = df_tests['Application:'].fillna('')
    df_tests = df_tests[df_tests['Application:'].str.startswith('Test:')]
    tests = df_tests['Anton Paar RheoCompass™ V1.22.0.0'].tolist()
    return [{'label': test, 'value': i} for i, test in enumerate(tests)]

@callback(
    Output('data-table', 'data'),
    Output('data-table', 'columns'),
    Output('plot-img', 'children'),
    Input('test-dropdown', 'value'),
    Input('color-picker', 'value'),
    Input('x-min', 'value'),
    Input('x-max', 'value'),
    Input('y-min', 'value'),
    Input('y-max', 'value'),
    Input('gridlines-toggle', 'value'),
    Input({'type': 'plot-label', 'index': ALL}, 'value'),
    Input('font-size', 'value'),
    Input('font-picker', 'value'),
    Input('marker-size', 'value'),
    State('sheet-dropdown', 'value'),
    State('upload-data', 'contents')
)
def update_output(selected_tests, selected_colors, x_min, x_max,y_min,y_max,gridlines_toggle, plot_labels, font_size, font_picker, marker_size, sheet_name, contents):
    if not selected_tests or sheet_name is None or contents is None:
        return [], [], ''

    if font_size is None:
        font_size = 12  # Set a default font size if None

    if font_picker is None:
        font_picker = 'Arial'  # Set a default font if None

    if marker_size is None:
        marker_size = 10  # Set a default marker size if None

    if selected_colors is None:
        selected_colors = color_options  # Set a default color list if None

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df_data = pd.read_excel(io.BytesIO(decoded), sheet_name=sheet_name, usecols=[
        'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']).dropna(how='any')

    # Define the value to split the DataFrame on
    split_value = 'Viscosity'
   
    # Group the DataFrame by the value to split on
    grouped = df_data.groupby((df_data['Unnamed: 4'] == split_value).cumsum())
   
    # Create a list of the tables of data for each test
    df_list = [group for name, group in grouped if len(group) >= 5]

    # # Ensure we have the same number of selected colors as tests
    selected_colors = selected_colors * (len(selected_tests) // len(selected_colors) + 1)

    # Prepare data and columns for the DataTable
    df_selected = pd.concat([df_list[i] for i in selected_tests])
    data = df_selected.to_dict('records')
    columns = [{'name': col, 'id': col} for col in df_selected.columns]

    # Plotting
    fig, ax = plt.subplots(figsize=(12,12))

    for i, test_index in enumerate(selected_tests):
        df_test = df_list[test_index]
        shear_rate = df_test['Unnamed: 3'].iloc[2:].astype(float)
        viscosity = df_test['Unnamed: 4'].iloc[2:].astype(float)

        ax.plot(shear_rate, viscosity, 'o-', label=f'{plot_labels[i] if i < len(plot_labels) else test_index}', color=selected_colors[i])

    ax.set_xlabel('Shear rate, $\gamma$/$s^{-1}$', fontname=font_picker, fontsize=font_size)
    ax.set_ylabel('Viscosity, $\eta$/Pa.s', fontname=font_picker, fontsize=font_size)
    ax.set_xscale('log')
    ax.set_yscale('log')

    ax.yaxis.set_major_formatter('{:.2f}'.format)
    ax.xaxis.set_major_formatter('{:.2f}'.format)
    ax.legend(prop={'family': font_picker, 'size': font_size})
    if 'show' in gridlines_toggle:
        ax.grid(True)
        ax.grid(which='minor', color='#EEEEEE')
    else:
        ax.grid(False)

    # Ensure x-axis and y-axis solid lines with ticks
    ax.tick_params(axis='both', which='both', direction='in', top=True, right=True, labelsize=font_size)
    ax.spines['top'].set_linewidth(1.5)
    ax.spines['right'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)

    if x_min is not None and x_max is not None:
        ax.set_xlim([x_min, x_max])
    if y_min is not None and y_max is not None:
        ax.set_ylim([y_min, y_max])
    plt.close(fig)

    # Convert plot to PNG image
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    encoded_image = base64.b64encode(output.getvalue()).decode('utf-8')
    img_src = f'data:image/png;base64,{encoded_image}'

    return data, columns, html.Img(src=img_src)

# Run the app then tyoe this http://127.0.0.1:2240/ to run in browser
if __name__ == '__main__':
    app.run(port=2240)